In [2]:
from minio import Minio
import os

# Initialize MinIO client
client = Minio(
    "127.0.0.1:9000",  # Example: "localhost:9000"
    access_key="minioaccesskey",
    secret_key="miniosecretkey",
    secure=False  # Set to True if using https
)

In [3]:
def upload_images_to_minio(local_folder, bucket_name):
    # Check if the bucket exists, create it if it does not
    if not client.bucket_exists(bucket_name):
        client.make_bucket(bucket_name)
        print(f"Bucket '{bucket_name}' created.")

    # Loop through the files in the local folder
    for filename in os.listdir(local_folder):
        local_file = os.path.join(local_folder, filename)

        # Skip directories
        if os.path.isdir(local_file):
            continue

        # Only upload image files (optional: modify this to filter specific types)
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
            try:
                # Upload the file to MinIO
                with open(local_file, "rb") as file_data:
                    client.put_object(bucket_name, filename, file_data, os.stat(local_file).st_size)
                print(f"Uploaded {filename} to {bucket_name}")
            except Exception as e:
                print(f"Failed to upload {filename}: {e}")


In [ ]:
local_root_folder = "./miniImageNet"
bucket_name = "image"
for root, _, files in os.walk(local_root_folder):
    print(root)
    # for file_name in files:
    #     if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff')):
    #         # Construct the local file path
    #         local_file_path = os.path.join(root, file_name)

    #         # Preserve the relative folder structure in the bucket
    #         relative_path = os.path.relpath(local_file_path, local_root_folder)
    #         bucket_file_path = relative_path.replace("\\", "/")  # Ensure consistent path separators

    #         # Call the upload function for each file
    upload_images_to_minio(root, bucket_name)

./miniImageNet
./miniImageNet\n01532829
Uploaded n01532829_10006.JPEG to image
Uploaded n01532829_10032.JPEG to image
Uploaded n01532829_10104.JPEG to image
Uploaded n01532829_10127.JPEG to image
Uploaded n01532829_1017.JPEG to image
Uploaded n01532829_1021.JPEG to image
Uploaded n01532829_1023.JPEG to image
Uploaded n01532829_10259.JPEG to image
Uploaded n01532829_10275.JPEG to image
Uploaded n01532829_1030.JPEG to image
Uploaded n01532829_1033.JPEG to image
Uploaded n01532829_10400.JPEG to image
Uploaded n01532829_1051.JPEG to image
Uploaded n01532829_10530.JPEG to image
Uploaded n01532829_10776.JPEG to image
Uploaded n01532829_10861.JPEG to image
Uploaded n01532829_10890.JPEG to image
Uploaded n01532829_1097.JPEG to image
Uploaded n01532829_110.JPEG to image
Uploaded n01532829_1106.JPEG to image
Uploaded n01532829_1114.JPEG to image
Uploaded n01532829_11167.JPEG to image
Uploaded n01532829_11202.JPEG to image
Uploaded n01532829_1124.JPEG to image
Uploaded n01532829_11266.JPEG to ima

In [11]:
import io
from minio import Minio

def read_image_from_minio(bucket_name, object_name):
    try:
        # Get the object from MinIO
        response = client.get_object(bucket_name, object_name)

        # Create an in-memory buffer and read the content into it
        image_data = io.BytesIO()
        for data in response.stream(32*1024):  # Read in chunks of 32KB
            image_data.write(data)

        # Reset pointer to the start of the buffer
        image_data.seek(0)

        # Optionally, you can open the image in memory using PIL (if needed)
        # from PIL import Image
        image = Image.open(image_data)

        # # You can now work with the image in memory, e.g., display it
        # image.show()  # Display the image

        return image  # Or return image_data if you need the raw bytes
    except Exception as e:
        print(f"Error reading image {object_name}: {e}")


In [5]:
object_name = "n02088094_1045.JPEG"  # The file path in MinIO

read_image_from_minio(bucket_name, object_name)


Error reading image n02088094_1045.JPEG: S3 operation failed; code: NoSuchKey, message: The specified key does not exist., resource: /image/n02088094_1045.JPEG, request_id: 180D6359A2BBD715, host_id: dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8, bucket_name: image, object_name: n02088094_1045.JPEG


In [ ]:
from qdrant_client import models, QdrantClient
from qdrant_client.models import PointStruct
from utils import FeatureExtractor
from PIL import Image
import os
import uuid  # To generate unique IDs for each point
qdrant_client = QdrantClient(url="http://localhost:6333")
collection_name = "img_embed"
feature_extractor = FeatureExtractor()
def populate_data():
        # Check if collection exists, if not create it
        try:
            qdrant_client.get_collection(collection_name)
        except Exception:
            qdrant_client.create_collection(
                collection_name=collection_name,
                vectors_config=models.VectorParams(
                    size=768,  # Adjust size based on your vector dimension
                    distance=models.Distance.DOT,  # Dot product distance
                ),
            )
        # Root directory for images
        root = "./miniImageNet"

        insert = True
        if insert:
            # Walk through the directory and process each image
            for dirpath, foldername, filenames in os.walk(root):
                for filename in filenames:
                    if filename.endswith(".JPEG"):
                        filepath = os.path.join(dirpath, filename)
                        image = Image.open(filepath).convert("RGB") 
                        # Extract the image embedding
                        image_embedding = feature_extractor(image)

                        # Create a unique ID for each point
                        point_id = str(uuid.uuid4())

                        # Create the payload as a dictionary (you can add more metadata here)
                        payload = {"filepath": filepath}
                        # print(payload, image_embedding.shape)
                        # Upload the point to the Qdrant collection
                        qdrant_client.upload_points(
                            # Ensure this matches your collection name
                            collection_name=collection_name,
                            points=[models.PointStruct(
                                id=point_id,  # Unique ID
                                vector=image_embedding.tolist(),  # Ensure this is a list
                                payload=payload  # Payload should be a dictionary
                            )],
                        )
                        # print(response)
        return
populate_data()


c:\Users\Admin\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\Admin\anaconda3\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
